<a href="https://colab.research.google.com/github/gahz8212/python/blob/master/%ED%95%9C%EA%B5%AD%EC%95%84%ED%8C%8C%ED%8A%B8%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#stacking(쌓는 모델)
#기본모델-여러개의 모델을 쌓아서 모델마다 학습함.
#메타모델-기본모델들에서 학습한 결과를 다시 한번 판단하는 모델로 학습함.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import numpy as np

In [20]:
# 1) 데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/csv/korea_apartment_sample.csv")


# 2) X(입력), y(타깃) 분리
X = df.drop("거래금액_만원", axis=1)
y = df["거래금액_만원"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# 4) 모델 학습 (랜덤포레스트)
model = RandomForestRegressor(n_estimators=300, random_state=42)
model.fit(X_train, y_train)

model2 = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    random_state=42
)
model2.fit(X_train, y_train)

model3 = LinearRegression()
model3.fit(X_train, y_train)


base_models=[
    ("lr",LinearRegression()),
    ("rf",RandomForestRegressor(n_estimators=300,random_state=42)),
    ("svr",make_pipeline(StandardScaler(),SVR(kernel='rbf',C=20))),
    ("lgbm",LGBMRegressor(n_estimators=500,random_state=42,learning_rate=0.5)) #부스팅모델 learning_rate는 C값과 비슷

           ]
meta_model=LGBMRegressor(
    n_estimators=300,
    learning_rate=0.5,
    random_state=42
)
from sklearn.ensemble import StackingRegressor
stack_model=StackingRegressor(estimators=base_models,final_estimator=meta_model,n_jobs=-1)
stack_model.fit(X_train,y_train)


y_pred = model.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)
y_pred4=stack_model.predict(X_test)



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 775
[LightGBM] [Info] Number of data points in the train set: 400, number of used features: 10
[LightGBM] [Info] Start training from score 65260.500000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [27]:
r2_1 = r2_score(y_test, y_pred)
r2_2 = r2_score(y_test, y_pred2)
r2_3 = r2_score(y_test, y_pred3)
r2_4 = r2_score(y_test, y_pred4)
print("랜덤포레스트 r2: ", r2_1)
print("라이트GBM r2: ", r2_2)
print("linear r2: ", r2_3)
print("stacking r2: ", r2_4)



랜덤포레스트 r2:  0.8697170262583864
라이트GBM r2:  0.8961431606541058
linear r2:  0.9141322336831011
stacking r2:  0.8377992316667899


In [30]:
mse1 = mean_squared_error(y_test, y_pred)
rmse1 = np.sqrt(mse1)

mse2 = mean_squared_error(y_test, y_pred2)
rmse2 = np.sqrt(mse2)

mse3 = mean_squared_error(y_test, y_pred3)
rmse3 = np.sqrt(mse3)

mse4 = mean_squared_error(y_test, y_pred4)
rmse4 = np.sqrt(mse4)

print("랜덤포레스트 rmse: ", rmse1)
print("라이트GBM rmse: ", rmse2)
print("linear rmse: ", rmse3)
print("stacking rmse: ", rmse4)



랜덤포레스트 rmse:  6133.67716100873
라이트GBM rmse:  5476.39333270323
linear rmse:  4979.5737064526775
stacking rmse:  6843.898117933239


In [33]:
price1 = model.predict([[1, 2025, 11, 87.366605, 10, 15, 500, 1, 2, 3]])
price2 = model2.predict([[1, 2025, 11, 87.366605, 10, 15, 500, 1, 2, 3]])
price3 = model3.predict([[1, 2025, 11, 87.366605, 10, 15, 500, 1, 2, 3]])
price4 = stack_model.predict([[1, 2025, 11, 87.366605, 10, 15, 500, 1, 2, 3]])
price1, price2, price3, price4

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-pa

(array([60896.81666667]),
 array([60731.6341193]),
 array([59517.49267737]),
 array([66926.57789827]))